In [4]:
import itertools
from z3 import *

# A Slice is a 2x2 grid representing Life state at a specific time.
Slice = list[list[ExprRef]]

def make_life(solver: Solver, grid_size: int, time_steps: int) -> list[Slice]:
    # Create a (time steps) x (grid size) x (grid size) array of symbolic variables.
    vars = [[[Int(f's_{t}_{i}_{j}') for j in range(grid_size)] for i in range(grid_size)] for t in range(time_steps)]

    for t in range(0, time_steps):
        for i, j in itertools.product(range(grid_size), range(grid_size)):
            # Each cell is either 0 or 1.
            solver.add(Or(vars[t][i][j] == 0, vars[t][i][j] == 1))

            if t == 0:
                continue

            # Life rules.
            count = 0
            for di, dj in itertools.product(range(-1, 2), range(-1, 2)):
                if di == dj == 0:
                    continue
                if 0 <= i + di < grid_size and 0 <= j + dj < grid_size:
                    count += vars[t - 1][i + di][j + dj]
            prev = vars[t - 1][i][j]
            next = vars[t    ][i][j]
            solver.add(Implies(And(prev == 1, next == 0), Or(count < 2, count > 3)))
            solver.add(Implies(And(prev == 1, next == 1), Or(count == 2, count == 3)))
            solver.add(Implies(And(prev == 0, next == 1), count == 3))
            solver.add(Implies(And(prev == 0, next == 0), count != 3))
    return vars

def print_model(model: ModelRef, state: list[Slice]) -> None:
    """Pretty print the model for the given state."""
    for t, s in enumerate(state):
        print(f"t = {t}")
        for i in range(len(s)):
            for j in range(len(s[0])):
                print(model[state[t][i][j]], end=" ")
            print()
        print()

def constrain(solver: Solver, s: Slice, on: set[tuple[int, int]]):
    """Constrain the given slice to be on at the given coordinates."""
    for i in range(len(s)):
        for j in range(len(s[0])):
            if (i, j) in on:
                solver.add(s[i][j] == 1)
            else:
                solver.add(s[i][j] == 0)


In [5]:
# Sanity check: 2x2 grid stays still
solver = Solver()
state = make_life(solver, grid_size=5, time_steps=4)
constrain(solver, state[0], set([(1, 1), (1, 2), (2, 1), (2, 2)]))
solver.check()
model = solver.model()
print_model(model, state)


t = 0
0 0 0 0 0 
0 1 1 0 0 
0 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 

t = 1
0 0 0 0 0 
0 1 1 0 0 
0 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 

t = 2
0 0 0 0 0 
0 1 1 0 0 
0 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 

t = 3
0 0 0 0 0 
0 1 1 0 0 
0 1 1 0 0 
0 0 0 0 0 
0 0 0 0 0 



In [8]:
# Sanity check: 1x3 rotation
solver = Solver()
state = make_life(solver, grid_size=3, time_steps=4)
constrain(solver, state[0], set([(0, 1), (1, 1), (2, 1)]))
solver.check()
model = solver.model()
print_model(model, state)

t = 0
0 1 0 
0 1 0 
0 1 0 

t = 1
0 0 0 
1 1 1 
0 0 0 

t = 2
0 1 0 
0 1 0 
0 1 0 

t = 3
0 0 0 
1 1 1 
0 0 0 



In [7]:
# Sanity check: glider
solver = Solver()
state = make_life(solver, grid_size=6, time_steps=6)
constrain(solver, state[0], set([(0, 1), (1, 2), (2, 0), (2, 1), (2, 2)]))
solver.check()
model = solver.model()
print_model(model, state)


t = 0
0 1 0 0 0 0 
0 0 1 0 0 0 
1 1 1 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 1
0 0 0 0 0 0 
1 0 1 0 0 0 
0 1 1 0 0 0 
0 1 0 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 2
0 0 0 0 0 0 
0 0 1 0 0 0 
1 0 1 0 0 0 
0 1 1 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 3
0 0 0 0 0 0 
0 1 0 0 0 0 
0 0 1 1 0 0 
0 1 1 0 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 4
0 0 0 0 0 0 
0 0 1 0 0 0 
0 0 0 1 0 0 
0 1 1 1 0 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 5
0 0 0 0 0 0 
0 0 0 0 0 0 
0 1 0 1 0 0 
0 0 1 1 0 0 
0 0 1 0 0 0 
0 0 0 0 0 0 



In [9]:
# Backwards: smiley
solver = Solver()
state = make_life(solver, grid_size=6, time_steps=4)
constrain(solver, state[-1], set([(1, 1), (1, 4), (3, 1), (3, 4), (4, 2), (4, 3)]))
solver.check()
model = solver.model()
print_model(model, state)


t = 0
1 0 1 0 0 1 
0 1 0 1 0 0 
1 1 0 1 1 1 
0 0 1 0 0 0 
0 0 1 1 1 1 
0 0 0 0 1 1 

t = 1
0 1 1 0 0 0 
0 0 0 1 0 1 
1 1 0 1 1 0 
0 0 0 0 0 0 
0 0 1 0 0 1 
0 0 0 0 0 1 

t = 2
0 0 1 0 0 0 
1 0 0 1 0 0 
0 0 1 1 1 0 
0 1 1 1 1 0 
0 0 0 0 0 0 
0 0 0 0 0 0 

t = 3
0 0 0 0 0 0 
0 1 0 0 1 0 
0 0 0 0 0 0 
0 1 0 0 1 0 
0 0 1 1 0 0 
0 0 0 0 0 0 

